In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
import ctypes
#from pynq.lib.arduino import ARDUINO_GROVE
#from pynq.lib import arduino
base = BaseOverlay('base.bit')
lib = MicroblazeLibrary(base.ARDUINO, ['gpio', 'spi', 'timer', 'uart','pmod_arduino'])

In [ ]:
%%microblaze base.ARDUINO
#include<uart.h>
#include<timer.h>
#include<spi.h>
#include<pmod_grove.h>
#include<iic.h>

int pec15Table[256];
int CRC15_POLY = 0x4599;
SPI_WidthType rdbuff=0x00;
SPI_ConfigType SpiConfig;
PIT_ConfigType pConfig;
UART_MemMapPtr tUART = UART2_BASE_PTR;
UART_ConfigType tUARTConfig;
UART_ConfigBaudrateType tBaud;

UART_MemMapPtr tUART1 = UART1_BASE_PTR;

void UARTRXCALLBACK()
{
	//UART_ReceiveWait(UART2_BASE_PTR, senduart, 1);
	rcvtemp=UART_GetChar(UART2_BASE_PTR);
	//UART_SendWait(tUART1, &rcvtemp, 1);
	cellno=rcvtemp>>1,status=rcvtemp&0b1;


	//UART_SendWait(tUART1, &cellno, 1);
	//UART_SendWait(tUART1, &status, 1);

	if(rcvtemp==0b10111000)
	{
		isframevalid=0;
		k=0;
	}
	else if((k>=0)&&(k<=17)&&(k==cellno))
	{
		CTRL[k]=status;
		k++;
	}
	else if(rcvtemp==0b01000111 && k==18)
	{
		k=-1;
		isframevalid=1;
		//UART_SendWait(tUART1, CTRL, 18);

		for(j=0;j<=17;j++)
			frame[j]=CTRL[j];

		PIT_TCTRL0 |= 1<<0;

	}
	else
	{
		k=-1;
		isframevalid=0;
	}


}


void init_PEC15_Table()
{
	int i, remainder, bit;
	for (i = 0; i < 256; i++)
	{
		remainder = i << 7;
		for (bit = 8; bit > 0; --bit)
		{
			if (remainder & 0x4000)
			{
				remainder = ((remainder << 1));
				remainder = (remainder ^ CRC15_POLY);
			}
			else
			{
				remainder = ((remainder << 1));
			}
		}
		pec15Table[i] = remainder & 0xFFFF;
	}
}
void readmatlab()
{

	CFGAR = (uint8_t [6]){CFGAR0,CFGAR1,CFGAR2,CFGAR3,CFGAR4,CFGAR5};
	CFGBR = (uint8_t [6]){CFGBR0,CFGBR1,CFGBR2,CFGBR3,CFGBR4,CFGBR5};

	sendcfgA=(uint8_t [12]){WRCFGA[0],WRCFGA[1],pec(WRCFGA, 2) >> 8,pec(WRCFGA, 2) & 0xFF,CFGAR0,CFGAR1,CFGAR2,CFGAR3,CFGAR4,CFGAR5,pec(CFGAR, 6) >> 8,pec(CFGAR, 6) & 0xFF};
	sendcfgB=(uint8_t [12]){WRCFGB[0],WRCFGB[1],pec(WRCFGB, 2) >> 8,pec(WRCFGB, 2) & 0xFF,CFGBR0,CFGBR1,CFGBR2,CFGBR3,CFGBR4,CFGBR5,pec(CFGBR, 6) >> 8,pec(CFGBR, 6) & 0xFF};
	txcountcfgr=12;

	for(i=0;i<3;i++);
	SSlow();
	SPI_TransferWait(SPI0_BASE_PTR, &rdbuff,sendcfgA,txcountcfgr);
	SShigh();
	for(i=0;i<3;i++);
	SSlow();
	SPI_TransferWait(SPI0_BASE_PTR, &rdbuff,sendcfgB,txcountcfgr);
	SShigh();
    
	return;
}



void readcellvoltage()
{
	UART_ReceiveWait(UART2_BASE_PTR, senduart, 16);
	ctrlzz=senduart[0];
	int cellno=ctrlzz>>1,status=ctrlzz&0b1;

	if(status==1)
		CFGAR4|=(1<<(cellno-1));
	else
		CFGAR4&=~(1<<(cellno-1));

	CFGAR[4]=CFGAR4;
	sendcfgA=(uint8_t [12]){WRCFGA[0],WRCFGA[1],pec(WRCFGA, 2) >> 8,pec(WRCFGA, 2) & 0xFF,CFGAR0,CFGAR1,CFGAR2,CFGAR3,CFGAR4,CFGAR5,pec(CFGAR, 6) >> 8,pec(CFGAR, 6) & 0xFF};

	for(i=0;i<3;i++);
	SSlow();
	SPI_TransferWait(SPI0_BASE_PTR, &rdbuff,sendcfgA,txcountcfgr);
	SShigh();

	UART_SendWait(tUART1, senduart, 16);
	return;
}




void ConfigInit()
{
	init_PEC15_Table();


	SpiConfig.sSettings.bPinAsOuput=0;
	SpiConfig.sSettings.bBidirectionModeEn=1;
	SpiConfig.sSettings.bIntEn=0;
	SpiConfig.sSettings.bTxIntEn=0;
	SpiConfig.u32BitRate = 115200;
	SpiConfig.u32BusClkHz = DEFAULT_SYSTEM_CLOCK;
	SpiConfig.sSettings.bMasterMode=1;
	SpiConfig.sSettings.bClkPhase1=0;
	SpiConfig.sSettings.bClkPolarityLow=0;
	SpiConfig.sSettings.bShiftLSBFirst=0;


	UART_Init(tUART, &tUARTConfig);
	UART_SetBaudrate(tUART, &tBaud);
	UART_Init(tUART1, &tUARTConfig);
	UART_SetBaudrate(tUART1, &tBaud);

	UART_SetCallback(tUART, UARTRXCALLBACK);
	return;
}